In [1]:
from requests import get
from requests.auth import HTTPBasicAuth
from IPython.core.interactiveshell import InteractiveShell
from pandas import DataFrame, read_sql_query
import pymysql
from sqlalchemy import create_engine
from decouple import config 
from dotenv import load_dotenv

from core.settings import setting



InteractiveShell.ast_node_interactivity = "all"

In [2]:
oauth = HTTPBasicAuth(setting.email_comcare,setting.password_comcare)

In [3]:
pay = {'xmlns':setting.xmlns, 'limit':setting.limit}
forms = get(setting.form_base_url,auth=oauth,params=pay).json()
__objects = forms["objects"]
__meta = forms["meta"]
print(__meta)

while __meta["next"]:
    __forms = get(f"""{setting.form_base_url}{__meta['next']}""",auth=oauth).json()
    __objects += __forms["objects"]
    __meta = __forms["meta"]

len(__objects)

{'limit': 1000, 'next': '?xmlns=http%3A%2F%2Fopenrosa.org%2Fformdesigner%2FDD8FE78F-C33C-4D7D-A63B-AEDFF3A85279&limit=1000&offset=1000', 'offset': 0, 'previous': None, 'total_count': 13252}


13252

In [4]:
pay_cases = {'type':setting.types,'limit':setting.caselimit}
form_cases = get(setting.case_base_url,auth=oauth,params=pay_cases).json()
__objects_cases = form_cases["objects"]
__meta_cases = form_cases["meta"]
print(__meta_cases)

while __meta_cases["next"]:
    __form_cases = get(f"""{setting.case_base_url}{__meta_cases['next']}""",auth=oauth).json()
    __objects_cases += __form_cases["objects"]
    __meta_cases = __form_cases["meta"]

len(__objects_cases)


{'limit': 5000, 'next': '?type=muso_household_2022&limit=5000&offset=5000', 'offset': 0, 'previous': None, 'total_count': 21196}


21196

In [5]:
beneficiare_direct = DataFrame(list(
    map(
        lambda k: {
            "userID":k['metadata']['userID'],
            "username":k['metadata']['username'],
            "timeEnd": k['metadata']['timeEnd'],
            "case_id": k['form']['case']['@case_id'],
            "houseold_number_2022": k['form']['household_number'],
            "db_name": k['form']['nom'],
            "db_first_name": k['form']['prenom']
        } 
        ,__objects
    )
))

beneficiare_direct.drop_duplicates('case_id',inplace=True)
beneficiare_direct.reset_index(drop=True)
#beneficiare_direct.case_id.value_counts()

beneficiare_direct.case_id.count()

,userID,username,timeEnd,case_id,houseold_number_2022,db_name,db_first_name
0,bfe6fcc7ba4f4c73899c3641fe40d905,4suzanne,2022-03-23T13:32:01.602000Z,6b99c059-c3c2-41c1-8638-90e72fac7511,3,Cesar,Jeancely
1,bfe6fcc7ba4f4c73899c3641fe40d905,4suzanne,2022-03-23T13:27:28.010000Z,2da5c226b6dc48298efce8f0ee5e509a,0,Macena,Linise
2,f9494c9ae5624f019566be84eddfd711,mlouissaint,2022-03-23T14:16:34.265000Z,c692786c72094fb49f1db46e6f833512,1,RAPHAEL,Mina
3,f9494c9ae5624f019566be84eddfd711,mlouissaint,2022-03-23T14:16:03.952000Z,639beb35be704ca0bba76e689cf86fd1,2,GUILLAUME,Marie Michel
4,f9494c9ae5624f019566be84eddfd711,mlouissaint,2022-03-23T14:15:20.707000Z,1e7139c8dd6d411a8179d67d3916cc0f,0,LAROSE,Marie Dominique
...,...,...,...,...,...,...,...
12428,02a25cddec634621822ccaa4657f8246,mcarthur,2022-02-16T20:41:26.240000Z,88881743-96fe-4e4b-b9e8-5bd30490ad8f,0,Diaquot,Widlet
12429,02a25cddec634621822ccaa4657f8246,mcarthur,2022-02-16T20:34:11.744000Z,7f6fc269-d750-44e1-b4cf-7446a14960a8,0,Darisma,John Evens
12430,02a25cddec634621822ccaa4657f8246,mcarthur,2022-02-16T20:26:50.145000Z,c2792c70-6b09-4f69-b2d4-69cc4498ed10,0,Guerrelus,Sony
12431,02a25cddec634621822ccaa4657f8246,mcarthur,2022-02-16T20:02:48.005000Z,724681f8-f8bd-4fb4-a0b4-c20c1c0b82d0,0,Vincent,Lovena


12433

In [6]:
beneficiare_indirect = DataFrame(list(
    map(
        lambda k: {
            "userID": k['user_id'],
            "ib_case_id": k['case_id'],
            "age": k['properties']['age'],
            "sexe": k['properties']['sexe'],
            "arv": k['properties']['arv'],
            "test": k['properties']['test'],
            "ib_case_id": k['case_id'],
            "parent_case_id": k['indices']['parent']['case_id'],
            "relationship": k['indices']['parent']['relationship'],
        } 
        ,__objects_cases
    )
))

beneficiare_indirect
beneficiare_indirect.ib_case_id.count()



,userID,ib_case_id,age,sexe,arv,test,parent_case_id,relationship
0,02a25cddec634621822ccaa4657f8246,2d6e3a4b-5a01-4f4f-9a69-b075aadac39f,3,1,,,b58699f1-47c2-4b05-ac0d-9a3034328010,child
1,8fd7f28e6c31484993c3153565feb429,ba32857a-7735-47b5-b9a6-abeb57b99ea9,3,2,non,non,cd1e58fb9bca47509b58e1e0c51e47e3,child
2,ece16e9f4d2c468e961d2b56ff621a2e,a253548c-4fbb-4ab2-b316-e2584e21c212,17,2,,,8b1013b7-287c-48ef-b123-f5bc451b7de9,child
3,8fd7f28e6c31484993c3153565feb429,36be1e5d-f2bf-4ec3-b35f-5e56e88aa8b6,17,1,non,non,6eb7b97bf219409c843d64f23f677c6b,child
4,ece16e9f4d2c468e961d2b56ff621a2e,0d34b807-6d7e-4190-9680-53bbf372df1a,11,2,,,995a88f0-2578-4c9d-a91c-57399e82557a,child
...,...,...,...,...,...,...,...,...
21191,f9494c9ae5624f019566be84eddfd711,2268bd65-ce35-444c-accc-d118356e3ba1,16,2,non,non,639beb35be704ca0bba76e689cf86fd1,child
21192,f9494c9ae5624f019566be84eddfd711,5e1e3e51-a084-4f75-809c-f7a979839403,8,1,non,non,c692786c72094fb49f1db46e6f833512,child
21193,bfe6fcc7ba4f4c73899c3641fe40d905,9be31889-bd36-4c3c-af83-32dbf0311fda,14,2,non,non,6b99c059-c3c2-41c1-8638-90e72fac7511,child
21194,bfe6fcc7ba4f4c73899c3641fe40d905,1d578f17-f88f-4f9b-8b54-e4056d21aba4,3,2,non,non,6b99c059-c3c2-41c1-8638-90e72fac7511,child


21196

In [7]:
load_dotenv()
# get the environment variables needed
USER= config('USRCaris')
PASSWORD= config('PASSCaris')
HOSTNAME= config('HOSTCaris')
DBNAME= config('DBCaris')

# get the engine to connect and fetch
engine = create_engine(f"mysql+pymysql://{USER}:{PASSWORD}@{HOSTNAME}/{DBNAME}")

query = f'''
SELECT
	au.username,
	au.email,
	count(*) as qty
FROM
	muso_household_2022 mh
	LEFT JOIN auth_users au ON mh.created_by = au.id
	group by au.username, mh.id_patient
'''

muso_hiv = read_sql_query(query,engine,parse_dates=True)
# get the test excel file from Query

# close the pool of connection
engine.dispose()


True

In [8]:
result_hivmuso = muso_hiv.groupby(['username','email']).count()
result_hivmuso.loc['Total']= result_hivmuso.qty.sum()
result_hivmuso.reset_index(inplace=True)
result_hivmuso


,index,qty
0,"(Ernstbelabre, ernst.belabre@carisfoundationin...",399
1,"(Gustave, firma.gustave@carisfoundationintl.org)",1049
2,"(June007, jeanrony.valbrun@carisfoundationintl...",979
3,"(despeignes, roselord.despeignes@carisfoundati...",342
4,"(jfodney17, fodney.joseph@carisfoundationintl....",895
5,"(josy22, josy.serafils@carisfoundationintl.org)",510
6,"(judesonvOLMAR, judeson.volmar@carisfoundation...",384
7,"(marcakernstdubois, kernst.dubois@carisfoundat...",703
8,"(suzanne, suzanne.clerge@carisfoundationintl.org)",121
9,"(viola, viola.tombeau@carisfoundationintl.org)",351
